# vnCDR sketch with cirq


This notebook shows the skeleton for a working variable noise Clifford data regression (vnCDR) implementation for a given circuit. The vnCDR training set construction is completed using cirq simulators and virtual machines. A single instance of a random quantum circuit is used to explore performance relative to Clifford data regression (CDR) and ZNE with various functional extrapolations. 

First, let’s review the vnCDR and CDR methods to remind ourselves of how these methods work and the elements necessary. vnCDR is the more general setting and is also expected to provide better mitigated results on average, so we will focus on the formalism for this method when presenting the necessary steps. 

## vnCDR 
vnCDR is a data driven error mitigation method. That means data from many quantum circuits is used to mitigate the effect of noise produced by one circuit of interest. vnCDR uses data from circuits which are classically simulable evaluated both classically (using a simulator) and on a real device at various levels of hardware noise. This data set is then used to train an ansatz mapping the noisy to exact data. The trained ansatz is used to mitigate the effect of the noise on the circuit of interest evaluated at different hardware noise levels. 

More formally, let $U$ be a quantum circuit which we are interested in, $|0 \rangle$ its initial state, and $X$ an observable of interest. We want to estimate the expectation value $\mu = \langle 0|U^{+} X U |0 \rangle$ from measurements on a noisy quantum device. We can mitigate the effect of noise using the following procedure:

- (Clifford data) Choose a set of circuits $\mathcal{S} = \{V_i\}_{i=1}^m$ based on $U$ which will be used to form the training set $\mathcal{T}$ in step 3. The circuits in $\mathcal{S}$ must be efficient to simulate classically, which is ensured by constructing them primarily from Clifford gates. The number of non-Cliffords used is denoted by $N$. Note that $N$ is assumed to be a constant parameter here, so the simulations are classically tractable. 

    
- (Noise data) Choose a set of noise levels $\mathcal{C}=\{c_0,c_1,\dots,c_n\}$ where $1 = c_0 < c_1 < \dots < c_n$ which will be used to form the training set $\mathcal{T}$ in step 3. If the noise is characterized by a parameter $\epsilon$ then running the device with noise level $c_j$ means the new parameter is $c_j\epsilon$.
    
- (Training set)  For each of the $m$ circuits $V_i$ in $\mathcal{S}$ and $n+1$ noise levels $c_j\in \mathcal{C}$, produce an estimate of the observable expectation called $x_{i, j}$. Also, for each of the $m$ circuits compute $y_i = \langle 0|V_i^{+} X V_i |0 \rangle$ using a classical simulation. The training set $\mathcal{T}$ is then defined as $\mathcal{T} = \{(\vec{x}_i, y_i)\}$ where $\vec{x}_i = (x_{i,0},\dots,x_{i,n})$ is the vector of noisy estimates originating from the $i^\text{th}$ circuit.

- (Learning) Learn a function $f : \mathbb{R}^n \to \mathbb{R}$ that takes a set of noisy estimates at the $n+1$ different noise levels and outputs an estimate for the noise-free value. Specifically, we take the linear ansatz $g : \mathbb{R}^n \times \mathbb{R}^n \to \mathbb{R}$,
\begin{equation}
           g(\vec{x}; \vec{a}) = \vec{a}\cdot\vec{x}\
\end{equation}
    We use least-squares regression on the dataset $\mathcal{T}$ to pick optimal parameters $\vec{a}^*$, i.e.,
    \begin{align}
        \vec{a}^* = \underset{\vec{a}}{\text{argmin}} \sum_{i=1}^m \left(y_i - g(\vec{x}_i; \vec{a})\right)^2\,,
    \end{align}
    so that we expect $f(\vec{x}) = g(\vec{x}; \vec{a}^*)$ to output a good estimate for the noise-free expected value given a vector of noisy ones.
    
- (Correction) Use the estimate $\hat{\mu} = \vec{a}^*\cdot\vec{\mu}$, where $\vec{\mu} = (\hat{\mu}_0,\dots,\hat{\mu}_n)$ is comprised of the $n+1$ noisy expectations for the original circuit.

## CDR 

There are two important distinctions between CDR and vnCDR. CDR does not use multiple hardware noise levels. Therefore there is not a vector of data for each circuit in the training set just a singular point. This also leads to the trianed ansatz being different. In the case of CDR the training set consisting of the noisy evaluated trianing circuits and their exact values are used to train a linear ansatz: 
\begin{equation}
           g(x; a, b) = ax + c.
\end{equation}

## Important features

Two important considerations when implementing vnCDR and CDR are how hardware noise is increased and how the circuits in the training set are constructed from the circuit of interest. In this sketch we present the simplest form of training set construction and hardware noise level increase. 

To construct the trianing set of classically simulable circuits here we randomly select a certain number of the non-Clifford gates in our circuit of interest and project them to their nearest Clifford gate, leaving $N$ non-Cliffords. This process is repeated $M$ times, giving a training set of size $M$. 

Noise is incrased by inserting identites formed of repeated CNOT gates. This is done using the unitary folding method provided in the mitiq package. 

The perforance of the mitigation is expected to converge with the number of non-Clifford gates in the training set circuits ($N$) and the number of circuits in the training set ($M$).



## Imports

In [3]:
import numpy as np
from sympy import *
import cirq
import scipy as sci
import matplotlib.pyplot as plt
import scipy.sparse as scs
from mitiq.mitiq_qiskit.conversions import to_qiskit, from_qiskit
from typing import Callable, Optional
from functools import wraps
from mitiq._typing import QPROGRAM
import numpy as np
from sklearn import linear_model
from operator import itemgetter
import random
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne.zne import execute_with_zne
from mitiq.zne.inference import LinearFactory, RichardsonFactory, PolyFactory, ExpFactory
from functools import partial
from qiskit.compiler import transpile
from cirq.optimizers import ConvertToCzAndSingleGates, MergeSingleQubitGates

## Random Quantum Circuit
In order to test the performance of vnCDR and CDR we need a test case to run on. Random quantum circuits (RQC) are a good option to use due to their generality. Below is the code to construct random quantum circuits composed of some gates with a fixed, alternating CNOT structure.


In [4]:
def random_circuit(
    qubits: int, 
    depth: int
)-> QPROGRAM:
    '''Function to generate a random quantum circuit in cirq. The circuit is based on the hardware efficient ansatz,
    with alternating CNOT layers with randomly selected single qubit gates in between.
    
    Args: 
        qubits: number of qubits in circuit. 
        depth: depth of the RQC.
        
    Returns: 
        cirquit: a random quantum circuit of specified depth.
        
'''
    # Get a rectangular grid of qubits.
    qubits = cirq.GridQubit.rect(qubits, 1)
    print(qubits)
    # Generates a random circuit on the provided qubits.
    circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(
        qubits=qubits, depth=depth, seed=0)
    circuit.append(cirq.measure(*qubits, key='z'))
    return(circuit)

## Functions used in CDR
Here we begin the code that is used in the CDR method. First up we have a function that takes any Qiskit circuit and compiles it to minimise depth, as well as decomposing it into a gate set where the only non-Clifford gate is the Rz rotation gate.

In [33]:
def TrigSolver(
    matrix: np.ndarray
)-> list:
    '''Decomposes U3(theta, phi, gamma) single qubit matrix into Rz(phi +pi)Rx(pi/2)Rz(theta + pi)Rx(pi/2)Rz(gamma) gate sequence. Used to 
       transalte the circuit of interest into the desired gate set for Clifford Projection.
       
    Args:
        matrix: U3 matrix to be decomposed.
    
    Returns:
        list of angles: [theta, phi, gamma]
    '''
    theta = Symbol('theta',real = True)
    phi = Symbol('phi',real = True)
    gamma = Symbol('gamma',real = True)
    phase = Symbol('phase',real = True)
    e1 = Eq(exp((1j*phase))*cos(theta/2)-matrix[0,0])
    e2 = Eq(exp((1j*phase))*(-exp((1j*gamma))*sin(theta/2))-matrix[0,1])
    e3 = Eq(exp((1j*phase))*exp((1j*phi))*sin(theta/2)-matrix[1,0])
    e4 = Eq(exp((1j*phase))*exp((1j*(gamma+phi)))*cos(theta/2)-matrix[1,1]) 
    [a1,a2,a3,a4] = np.random.rand(4)*2*np.pi
    try:
        soln = nsolve((e1,e2,e3,e4),(theta, phi, gamma, phase), (a1,a2,a3,a4))
        [theta, phi, gamma, phase] = [re(soln[0,0]),re(soln[1,0]), re(soln[2,0]), re(soln[3,0])]
    except:
        TrigSolver(matrix)
    return(theta, phi, gamma)
    

In [106]:
def pauli_gate_to_u3(
    gate: cirq.Gate
)-> cirq.Gate:
    '''Function to decompose a single Pauli rotation gate into U3 form composed of Rx(pi/2) pulses and Rz rotation gates
    
    Args:
        gate: Pauli rotation gate.
    
    Returns: 
        gate_sequence: Sequence of rotation gates that lead to the above pauli rotation, decomposed into Rx(pi/2) pulses
        and Rz gates of three angles.
    '''
    gates = []
    name = gate[0]
    angle = gate[1]
    
    if name == cirq.X:
        gates.append(cirq.rz(np.pi/2))
        gates.append(cirq.rx(np.pi/2))
        gates.append(cirq.rz(angle*(np.pi/2)+np.pi))
        gates.append(cirq.rx(np.pi/2))
        gates.append(cirq.rz(np.pi/2))
        
    elif name == cirq.Y:
        gates.append(cirq.rz(np.pi))
        gates.append(cirq.rx(np.pi/2))
        gates.append(cirq.rz(angle*(np.pi/2)+np.pi))
        gates.append(cirq.rx(np.pi/2))
        
    elif name == cirq.Z:
         gates.append(cirq.rz(angle*(np.pi/2)))
            
    return(gates)
        

In [107]:
def CircuitTranslation(
    circuit: QPROGRAM
)-> QPROGRAM:
    """Decomposes cirq circuit object into Rz, Rx(pi/2) (sx), X and CNOT gates. 
    
    Args:
        circuit: original circuit of interest assumed to be cirq circuit object.
        qubits: number of qubits in the circuit.
        
    Returns: 
        circuit_new: new circuit compiled and decomposed into the above gate set.
    """
    depth = len(circuit)
    nqubits = len(circuit.all_qubits())
    qubits = circuit.all_qubits()
    circuit_new = cirq.GridQubit.rect(nqubits, 1)
    ConvertToCzAndSingleGates().optimize_circuit(circuit = circuit)
    MergeSingleQubitGates().optimize_circuit(circuit=circuit)
    for operation in circuit.all_operations():
        # finding and decomposing the single qubit gates into Rz and Rx(pi/2) gates:
        qubit = operation.qubits
        if len(qubit) == 1:
            gates = cirq.optimizers.single_qubit_matrix_to_pauli_rotations(operation.gate._unitary_())
            print(gates)
            for gate in gates: 
                decomposed_gate = pauli_gate_to_u3(gate)
                for r_gate in decomposed_gate:
                    print(r_gate)
                    circuit_new.append(r_gate(qubit))
        else:
            cirq
                
    return(circuit)

It should be noted that the gates that are involved in this decomposition are soon to be made the basis gate set for all IBM computers. Lets have a look at the circuits we have constructed and how the decomposition affects our circuit.

In [108]:
num_qubits = 4
layers = 2
circuit = cirq.circuits.Circuit(random_circuit(num_qubits,layers))

[cirq.GridQubit(0, 0), cirq.GridQubit(1, 0), cirq.GridQubit(2, 0), cirq.GridQubit(3, 0)]


In [109]:
circuit_new = CircuitTranslation(circuit)
# print(circuit_new.qid_shape())
# for operation in circuit_new.all_operations():
#     print(TrigSolver(operation.gate._unitary_()))


[(cirq.X, 1.0)]
[cirq.rz(np.pi*0.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*1.5), cirq.rx(np.pi*0.5), cirq.rz(np.pi*0.5)]
Rz(0.5π)


AttributeError: 'tuple' object has no attribute 'dimension'

How does the decomposition change the depth?

In [ ]:
print('old depth', circuit.depth())
print('new depth', circuit_new.depth())

The decomposition has increased the depth by some amount. This means that when implementing the circuit on a real device or noisy simulator the raw data from the decomposed circuit is likely to be worse than that from the original.

The next function takes a Qiskit circuit and extracts the information stored in the circuit, returning that information as an array. This makes many necessary steps in the CDR methods easier to implement, like completing gate substitutions for example.

In [ ]:
def CircToArray(
    circ: QPROGRAM
)-> np.ndarray:
    '''Function to return the order of gates, their names and paramters in a more managable data structure than a Qiskit
    quantum circuit.
    
    Args:
        circ (QPROGRAM): Qiskit circuits (deomposed).
        
    Returns:
    data (np.ndarray): np.array([order], [names], [parameters], [qubits], [cbits])
    where order is the order of the gates from 0 to depth, names are the names of the gates, parameters are the paramters 
    specifying the gates and qubits and cbits are the qubits and classical bits on which they act.
    qubits (int): number of qubits.
    clbits (int): number of classical bits.
    '''
    order=[]
    gates_list = []
    qubits_list = []
    cbit_list = []
    parameters_list = []
    qubits = len(circ.qubits)
    clbits = len(circ.clbits)
    #loop through and construct arrays:
    for i in range(len(circ.data)):
        order.append(i)
        gate = circ.data[i][0]
        name = gate.name
        if name == 'cx':
            qubit = [circ.data[i][1][0].index, circ.data[i][1][1].index]
            parameters =None
            gates_list.append(name)
            qubits_list.append(qubit)
            cbit_list.append(None)
            parameters_list.append(parameters)
        elif name =='rz':
            parameters = gate.params[0]
            gates_list.append(name)
            qubit = circ.data[i][1][0].index
            qubits_list.append(qubit)
            cbit_list.append(None)
            parameters_list.append(float(parameters))
        elif name =='rx':
            parameters = np.pi/2
            gates_list.append('rx')
            qubit = circ.data[i][1][0].index
            qubits_list.append(qubit)
            cbit_list.append(None)
            parameters_list.append(parameters)
        elif name =='x':
            parameters = None
            gates_list.append(name)
            qubit = circ.data[i][1][0].index
            qubits_list.append(qubit)
            cbit_list.append(None)
            parameters_list.append(parameters)
        elif name =='measure':
            parameters = None
            gates_list.append(name)
            qubit = circ.data[i][1][0].index
            cbit = circ.data[i][2][0].index
            qubits_list.append(qubit)
            cbit_list.append(cbit)
            parameters_list.append(parameters)
    data = np.array([order, gates_list,  parameters_list, qubits_list, cbit_list])
    return data, qubits, clbits
        

We also need a function that takes this array and gives back a Qiskit quantum circuit, this function is below.

In [ ]:
def ArrayToCirc(
    data: np.ndarray,
    qubits: int,
    clbits: int) -> QPROGRAM:
    ''' Function that takes the data array containing all the circuit data and turns it into a quantum circuit.
    
    Args:
        data: array containing circuit data np.array([order], [names], [parameters], [qubits], [cbits]).
        qubits: number of qubits.
        clbits: number of classical bits.
        
    Returns:
        circ: QPROGRAM (Qiskit quantum circuit)
    
    '''
    gates_list= data[1,:]
    parameters_list = data[2,:]
    qubits_list = data[3,:]
    cbit_list = data[4,:]
    circ = QuantumCircuit(qubits,clbits)
    for i in range(len(gates_list)):
        gate = gates_list[i]
        parameter = parameters_list[i]
        qubit = qubits_list[i]
        cbit = cbit_list[i]
        if gate =='rz':
            circ.rz(parameter, qubit)
        if gate=='rx':
            circ.rx(parameter, qubit)
        if gate=='cx':
            circ.cx(qubit[0], qubit[1])
        if gate=='x':
            circ.x(qubit)
        if gate=='measure':
            circ.measure(qubit, cbit)
    return(circ)
        

## Clifford based functions
Here are the functions that check if a gate in our circuit is a Clifford gate and then use this information to complete projections and construct a training set.

In [ ]:
def CliffordChecker(
    ang: float
)-> bool:
    '''Function to check if a given angle is Clifford.
    
    Args:
        ang: rotation angle in the Rz gate.
        
    Returns:
        bool: True / False for Clifford or not. 
    '''
    cliff_angs = [np.pi/2, np.pi, np.pi*3/2, 2*np.pi]
    diff_list=[]
    for cliff_ang in cliff_angs:
        diff_list.append(abs(abs(ang)-cliff_ang)%(2*np.pi))
    if any(np.array(diff_list)<=10**(-8)) == True:
        return True
    else:
        return False
    
# Vectorize function so it can take arrays of angles as its input.
CliffordChecker = np.vectorize(CliffordChecker)

In [ ]:
def ClosestClifford(
    ang: float
)-> float:
    '''Function to take angle and return the nearest Clifford angle note the usage of this function is vectorized
    so it takes and returns arrays.
    
    Args:
        ang: angle in Rz gate.
        
    Returns:
        Clifford angle: closest clifford angle.
    '''
    cliff_angs = [np.pi/2, np.pi, np.pi*3/2, 2*np.pi]
    diff_list=[]
    for cliff_ang in cliff_angs:
        diff_list.append(abs((ang)%(2*np.pi)-cliff_ang))
    min_1 = min(diff_list)
    # need to check that there are not two min values in list
    diff_list_copy = diff_list.copy()
    diff_list_copy.remove(min_1)
    min_2 = min(diff_list_copy)
    # if just one min value, return the corresponding nearest cliff.
    if abs(min_1-min_2) > 10**(-8):
        return (cliff_angs[diff_list.index(min(diff_list))])
    # if two min values (ie two cliff gates equidistant) randomly choose the cliff gate to return.
    else:
        index_list = [diff_list.index(min_1), diff_list_copy.index(min_2)]
        index = random.choice(index_list)
        return(cliff_angs[index])

# vectorize so function can take array of angles.
ClosestClifford = np.vectorize(ClosestClifford)

Below is the function that takes one circuit of interest and constructs a trianing set by randon projections. This means that a given number of the non-Clifford gates in the circuit are selected and projected to their closest Clifford gate leaving N non-Cliffords. This process is repeated, producing a set of randomly projected Clifford dominated circuits.

In [ ]:
def TrainingCircuitsConstructor(
    data: np.ndarray,
    size: int ,
    N: int
)-> np.ndarray:
    """Function that takes the original circuit and creates a training set with a given number of non-cliffords and total size.
    Args:
        data: data of the circuit of interst.
        size: number of Clifford projected circuits to return (size of training set).
        N: Number of non-Clifford gates to leave in each ciruit in the trianing set.
    
    Returns:
        circ_data: list of data arrays for projected circuits.
    """
    circ_data=[]
    mask_rz = data[1,:] == 'rz'
    rz_circ_data = data[:,mask_rz]
    mask_not_rz = data[1,:] != 'rz'
    not_rz_circ_data = data[:,mask_not_rz]
    mask_non_cliff = CliffordChecker(rz_circ_data[2,:]) == False
    rz_non_cliff = rz_circ_data[:,mask_non_cliff]
    mask_cliff = CliffordChecker(rz_circ_data[2,:]) == True
    rz_cliff = rz_circ_data[:,mask_cliff]
    tot_n_non_cliff = len(rz_non_cliff[0])
    all_cliff = np.column_stack((not_rz_circ_data, rz_cliff))
    for s in range(size):
        random_columns = random.sample(np.arange(0,tot_n_non_cliff,1).tolist(),tot_n_non_cliff-N)
        rand_rz_non_cliff_proj = rz_non_cliff.copy()[:,random_columns]
        rand_rz_non_cliff_proj[2,:] = ClosestClifford(rand_rz_non_cliff_proj[2,:])
        new_circ = np.column_stack((all_cliff, rand_rz_non_cliff_proj))
        i = np.argsort(new_circ[0,:])
        final = new_circ[:,i]
        circ_data.append(final)
    return(circ_data)

In [ ]:
def NoiseIncrease(
    circ: QPROGRAM, 
    scale_factor: int
)-> QPROGRAM:
    '''Function to scale the noise in the same manner to all circuits - circuit of interest and circuits in trianing set.
    
    Args:
        circ: circuit object
        scale_factor: the scale factor with which to scale the noise level.
    Returns:
        folded_circ: the new circuit with increased noise.
    '''
    # Just going to put one example of noise increasing here, this needs to be generalised. 
    folded_circ = fold_gates_at_random( circ, scale_factor=scale_factor, fidelities={"single": 1.0,
                                       "CNOT": 0.99})
    return(folded_circ)

## Utility functions for Qiskit 
Here we define some functions that are important to extract the desired quantities from the Qiskit simulation/ real device data.

In [ ]:
def binarize(
    decimal: float, 
    L: int
)-> float:
    """Converts a decimal number <=2**L to binary. It is used to make the qiskit convention and the usual 
    description of the basis of an L-spin system compatible.
    
    Args:
        decimal: (float) Number in base 10 to be converted to binary.
        L: (float) Total number of qubits in the system.
    Returns:
        (string) Number described by the parameter 'decimal' in binary.
    """
    binario = np.zeros(L)
    bin = ''
    for i in range(L):
        binario[L - 1 - i] = decimal % 2
        decimal = decimal // 2
        bin = bin + str(binario.astype(int)[L - 1 - i])

    return bin[::-1]

In [ ]:
def expected_value(
    Obs: np.ndarray,
    state: dict
)-> float:
    """Calculates the expected value of a diagonal observable.
    
    Args:
        Obs: (numpy.array) Observable in the form of a diagonal matrix.
        state: (dict) Dictionary describing the state of the system on which the expected value of the observable is to
        be calculated.
        
    Returns:
        (float) Expected value <state|Obs|state>.
    """
    obs = sum([Obs[i, i] * list(state.values())[i] for i in range(np.shape(Obs)[0])])
    return obs


In [ ]:
def state_counts(
    counts: dict
)-> dict:
    """Expresses the result of the simulation in the usual way, that is, in the form of a dictionary whose values are
    the module squared of the components of the final state.
    
    Args:
        counts: (dict) Dictionary included in Qiskit result class that allows that allows to get the histogram
        data of an experiment.
        
    Returns:
        state (dict) Dictionary whose keys are the base elements of the L qubit system and whose values are the
        modulus of the corresponding squared amplitudes.
    """
    L = len(list(counts)[0])
    basis = {i: binarize(i, L) for i in range(2 ** L)}
    counts_order = [0 for i in range(2 ** L)]
    for i in range(len(basis)):
        for j in range(len(counts)):
            if list(basis.values())[i] == list(counts)[j][::-1]:
                counts_order[i] = list(counts.values())[j]
    normalization = 0
    for i in range(2 ** L):
        normalization += counts_order[i]

    state = {list(basis.values())[i]: counts_order[i] / normalization for i in range(2 ** L)}
    return state



In [ ]:
def Magnetization(
    j: float, 
    L: float
)-> sci.sparse.csc_matrix:
    """Generation of the local magnetization (sigma_z) operator.
    
    Args:
        j: (float) Label of the qubit on which its magnetization is to be calculated.
        L: (float) Total number of qubits in the system.
    Return:
        Hz (scipy.sparse.csc_matrix of dimension 2**L) Magnetization operator for qubit j in matrix form
    """
    sigmaz = scs.csc_matrix([[1, 0], [0, -1]])
    Id = scs.identity(2,format='csc')
    Hz = scs.csc_matrix((2**L, 2**L))
    Hzj = Id
    if j == 0:
        Hzj = sigmaz
    for i in range(L - 1):
        if i < j - 1:
            Hzj = scs.kron(Hzj, Id, format='csc')
        elif i == j - 1:
            Hzj = scs.kron(Hzj, sigmaz, format='csc')
        elif j - 1 < i < L - 1:
            Hzj = scs.kron(Hzj, Id, format='csc')
    Hz = Hz + Hzj
    return Hz


## Execute and Simulate functions
Here we define the functions that execute a given circuit on a 'real' device (in this case we use a simulated device with a given noise model), and execute a circuit on a exact simulator. These two functions are then used to build the training set data, consisting of exact and noisy expectation values.

Here the executor takes any circuit and runs it on a simulation of the IBMQ Bogota computer, with a simulated noise model. The float that is returned is the expectation value of the $\sigma_{z}$ observable at some position. 


In [ ]:
def executor(
    circ: QPROGRAM, 
)-> float:
    '''
    Function to run my noisey circuits, being treated as real backend. Measures <sigma_z> on first qubit,
    assuming all qubits are measured.
    
    Args: 
        circ: quantum circuit to be evaluated.        
    returns:
        <sigma_z>: returns the observable at the given position with simulated device noise.
    '''
    position = 1
    device = 'ibmq_bogota'
    qubits = len(circ.qubits)
    obs = Magnetization(position, qubits)
    try:
        real_device = provider.get_backend(device)
        coupling_map = real_device.configuration().coupling_map
        noise_model = aern.NoiseModel.from_backend(real_device)
    except:
        real_device = provider.get_backend(device)
        coupling_map = real_device.configuration().coupling_map
        noise_model = aern.NoiseModel.from_backend(real_device)
    job = execute(circ, Aer.get_backend('qasm_simulator'),
              optimization_level=0, shots=8192, noise_model=noise_model, 
                  basis_gates=noise_model.basis_gates, coupling_map=coupling_map)
    result = job.result()
    counts = result.get_counts()
    counts = {k.replace(' ', ''): v for k, v in counts.items()}
    state = state_counts(counts)
    sigma_z = expected_value(obs, state)
    return(sigma_z)

The simulator function is the same as the executor but returns exact results (up to shot noise).

In [ ]:
def simulator(
    circ: QPROGRAM, 
)-> float:
    '''Function to run my noisey circuits, being treated as real backend. Measures <sigma_z> on a given qubit specified
    by the position variable assuming all qubits are measured.
    
    Args: 
        circ: quantum circuit to be evaluated. 
        
    returns:
        <sigma_z>: returns the exact observable at the given position but includes shot noise.
    '''
    position = 1
    qubits = len(circ.qubits)
    obs = Magnetization(position, qubits)
    job = execute(circ, Aer.get_backend('qasm_simulator'),
              optimization_level=0, shots=8192)
    result = job.result()
    counts = result.get_counts()
    counts = {k.replace(' ', ''): v for k, v in counts.items()}
    state = state_counts(counts)
    sigma_z = expected_value(obs, state)
    return(sigma_z)

With these two functions the training set data can then be generated by passing the training circuits through the exector and simulator functions. Note that due to the currrent definition of these functions each observable will have its own training set used for correction. This means to correct each observable $M$ circuits will need to be evaluated classically and on the real device. In reality this might not be neccesary. One training set could be used to correct many observables (as long as the circuit of interest is not affected by their evaluation, ie many observables diagonal in the z basis could be corrected from the same training set potentially). 

However, in this tutorial we are only looking at one particular observable, $\sigma_{z}$ at some provided position, so the way the executor and simulator are defined here is sufficient.

In [ ]:
def TrainingSetGenerator(
    circ: QPROGRAM, 
    scale_factors: list, 
    M: int, 
    N: int
)-> np.ndarray:
    '''Function that takes a circuit of interest and the position of sigma_z observable and paramters to construct the training
    set, returns an array of the training data.
    
    Args: 
        circ: circuit of interest.
        scale_factors: scale factors to increase the noise.
        M: size of training set. 
        N: number of non-Cliffords per circuit in training set.
        
    Return:
        training_data: M by (scale_factors+1) array containing exact and noisy observables at different scale factors. 
    '''
    circ_data, qubits, clbits = CircToArray(circ)
    training_circ_data = TrainingCircuitsConstructor(circ_data, M, N)
    training_data = np.zeros((M, 1+len(scale_factors)))
    for i in range(len(training_circ_data)):
        train_circ = ArrayToCirc(training_circ_data[i], qubits, clbits)
        training_data[i,0] = simulator(train_circ)
        for s, scale_factor in enumerate(scale_factors):
            noise_inc_circ = NoiseIncrease(train_circ, scale_factor)
            training_data[i,s+1] = executor(noise_inc_circ)
    return(training_data)        

## Regression functions
Below we have the functions that take the training set produced above and fit it with the vnCDR and CDR ansatzes. The fitting is done using the package sklearn for ease of use, but this can be replaced if so desired.

In [ ]:
def vnCDR_mitigate(
    train_data: np.ndarray, 
    circ_data: np.ndarray
)->float:
    '''Function that uses the training data to perform a regression and return the best fit variables the ansatz used is 
    a simple linear model. 
    
    Args: 
        train_data: Data from Clifford circuits used to train the model. 
        circ_data: Data from actual circuit of interest which the trained model is then used to mitigate.
        
    Returns:
        corrected: mitigated value caluclated using the provided training set.
    '''
    reg = linear_model.LinearRegression(fit_intercept=False)
    
    X_train = np.array(train_data[:,1:])
    y_train = np.array(train_data[:,0])
    X_test = circ_data
    X_test.reshape(1,-1)
    reg.fit(X_train, y_train)
    corr = reg.predict([X_test])
    return corr[0]

Note below only one noise level is used in the mitigation, as required for CDR.

In [ ]:
def CDR_mitigate(
    train_data: np.ndarray, 
    circ_data: np.ndarray
)-> float:
    '''Function that uses the training data to perform a regression and return the best fit variables using oly one noise level.
    
    Args:
        train_data: Data from Clifford circuits used to train simple linear anstaz (only one noise level used here).
        circ_data: Data from actual circuit of interest which is mitigated by the trained model.
        
    Returns:
        corrected: mitigated observable.
    '''
    reg = linear_model.LinearRegression(fit_intercept=True)
    X_train = np.array(train_data[:,1]).reshape(-1,1)
    y_train = np.array(train_data[:,0]).reshape(-1,1)
    X_test = circ_data.reshape(1, -1)
    reg.fit(X_train, y_train)
    corr = reg.coef_[0]*circ_data+reg.intercept_[0]
    return corr[0]

Now all the functions have been defined we can generate the real data for the circuit of interest at various noise levels.

In [ ]:
def circuit_data(
    circ: QPROGRAM,
    scale_factors:list
)-> np.ndarray:
    '''Function that takes a circuit of interest and observable position and returns the observable at scaled noise values.
    
    Args:
        circ: circuit of interest.
        scale_factors: list of factors with which to scale the noise.
        
    Returns:
        circ_data: array of observables at increasing noise strength.
    '''
    circ_data = np.zeros((len(scale_factors)))
    for s, scale_factor in enumerate(scale_factors):
        noise_inc_circ = NoiseIncrease(circ, scale_factor)
        circ_data[s] = executor(noise_inc_circ)
    return circ_data
    

In [ ]:
def clifford_data_mitigation(
    circ: QPROGRAM, 
    scale_factors: list, 
    M: int, 
    N: int
)-> list:
    '''Function that takes a circuit and scale factors and returns the vnCDR and CDR mitigations as well as the raw result.
    
    Args:
        circ: quantum circuit of interest. 
        position: position of sigma_z observable.
        scale_factors: scale factors with which to increase the noise.
        M: size of the training set used in CDR and vnCDR. 
        N: number of non-Clifford gates per training set.
    
    Returns:
        vnCDR_mit: result mitigated by vnCDR. 
        CDR_mit: result mitigated by CDR.
        circ_data: raw noisy result.
    '''
    training_data = TrainingSetGenerator(circ,scale_factors, M, N)
    circ_data = circuit_data(circ, scale_factors)
    vnCDR_mit = vnCDR_mitigate(training_data, circ_data)
    CDR_mit =CDR_mitigate(training_data, circ_data[0])
    return vnCDR_mit, CDR_mit, circ_data[0], training_data
    

In [ ]:
def Abs_error(
    val_1: float, 
    val_2:float
)->float:
    '''Function to calculated the absolute error between two floats
    
    Args:
        val_1: noisy/ mitigated observable. 
        val_2: exact observable.
        
    Returns:
        absolute error.
    '''
    return abs(val_1-val_2)/abs(val_2)

Now having defined all these functions we can tie them together and compare the performance for the RQC constructed at the beginning of this notebook.

First lets define the parameters we will use, in order to do this we should check how many non-Clifford gates there are in our original decomposed circuit of interest:

In [ ]:
data, _, _ = CircToArray(circuit_new)
mask_rz = data[1,:] == 'rz'
rz_circ_data = data[:,mask_rz]
mask_not_rz = data[1,:] != 'rz'
not_rz_circ_data = data[:,mask_not_rz]
mask_non_cliff = CliffordChecker(rz_circ_data[2,:]) == False
rz_non_cliff = rz_circ_data[:,mask_non_cliff]
print('Number non-Cliffords:', len(rz_non_cliff[0]))

We need to use a smaller number than the above result and lets just assume we are only interested in the $\sigma_{z}$ of the 1st qubit, so lets take the following:

In [ ]:
scale_factors = [1.0,2.0,3.0,4.0]
M=30
N=6

Now lets execute the vnCDR and CDR mitigation methods. Note this may take some time due to the construction of the training set.

In [ ]:
print('Data regression underway')
vnCDR_mit, CDR_mit, noisy, training_data = clifford_data_mitigation(circuit_new, scale_factors[0:2],M,N)
print('Running exact simulation (with shot noise)')
exact = simulator(circuit_new)

In [ ]:
print('exact',exact)
print('noisy', noisy)
print('vnCDR mitigated', vnCDR_mit)
print('CDR mitigated', CDR_mit)

Lets have a look at the observables in the training set that are used to understand the noise. This is often an important step in investigating how effective the training set construction method is, and if the mitigation can be improved.

In [ ]:
noise_level=1
exact_observables = training_data[:,0]
noise_1_observables = training_data[:,noise_level]
plt.figure(1)
plt.hist(exact_observables, bins=10, alpha=0.7, label = 'exact')
plt.hist(noise_1_observables, bins=10, alpha = 0.7, label = 'noisy')
plt.legend()
plt.xlabel(r'$\sigma_{z}$')
plt.ylabel('frequency')
plt.show()
plt.figure(2)
plt.scatter(exact_observables, noise_1_observables)
plt.xlabel('exact')
plt.ylabel('noisy (1st noise level)')
plt.show()

In general a training set with a diverse range of both noisy and exact values is more effective and giving an accurate mitigation. These diverse training sets require more complex methods of construction that random projection.

We can also run ZNE with various fitting forms for a comparison:

In [ ]:
print('ZNE underway')
# method: scale noise by 1 and 2, then extrapolate linearly to the zero noise limit.
linear_fac = LinearFactory(scale_factors=scale_factors[0:2])

# method: scale noise by 1, 2 and 3, then evaluate the Richardson extrapolation.
richardson_fac = RichardsonFactory(scale_factors=scale_factors[0:3])

# method: scale noise by 1, 2, 3, and 4, then extrapolate quadratically to the zero noise limit.
poly_fac = PolyFactory(scale_factors=scale_factors, order=2)

# method: scale noise by 1, 2, 3, and 4, then extrapolate exponentially to the zero noise limit.
exp_fac = ExpFactory(scale_factors=scale_factors)

lin_zne = linear_fac.run(circuit_new, executor, scale_noise=partial(fold_gates_at_random, fidelities={"single": 1.0,"CNOT": 0.99}))
rich_zne = richardson_fac.run(circuit_new, executor, scale_noise=partial(fold_gates_at_random, fidelities={"single": 1.0,"CNOT": 0.99}))
poly_zne = poly_fac.run(circuit_new, executor, scale_noise=partial(fold_gates_at_random, fidelities={"single": 1.0,"CNOT": 0.99}))
exp_zne = exp_fac.run(circuit_new, executor, scale_noise=partial(fold_gates_at_random, fidelities={"single": 1.0,"CNOT": 0.99}))


Finally lets compare the results:

In [ ]:
print('Absolute errors:')
try:
    zne_lin = execute_with_zne(circuit_new, executor, factory=linear_fac)
    print('lin_zne error', Abs_error(zne_lin, exact))
except:
    zne_lin = None
try:
    zne_rich = execute_with_zne(circuit_new, executor, factory=richardson_fac)
    print('rich_zne error', Abs_error(zne_rich, exact))
except:
    zne_rich = None
try:
    zne_poly = execute_with_zne(circuit_new, executor, factory=poly_fac)
    print('poly_zne error', Abs_error(zne_poly, exact))
except:
    zne_poly = None
try:
    zne_exp = execute_with_zne(circuit_new, executor, factory=exp_fac)
    print('exp_zne error', Abs_error(zne_exp, exact))
except:
    zne_exp = None


print('vnCDR error', Abs_error(vnCDR_mit, exact))
print('CDR_mit error', Abs_error(CDR_mit, exact))
print('noisy error', Abs_error(noisy, exact))



Hopefully one can see that the results from Clifford based methods are superior. This may vary according to the noise model from the IBM computer, which does change. Lets explore how the noisy results are changing with increased noise level, this is always something important to look at to inform the best extrpolation method to use.

In [ ]:
circuit_data_example = circuit_data(circuit_new, scale_factors)

In [ ]:
plt.figure(1)
plt.scatter(scale_factors, circuit_data_example, label = 'noisy', color = 'blue')
plt.scatter([0], [exact], label = 'exact', color = 'black')
plt.scatter([0], [zne_lin], label = 'zne_lin', color = 'red', marker = '*')
plt.scatter([0], [vnCDR_mit], label = 'vnCDR', color = 'green', marker = '+')
plt.scatter([0], [CDR_mit], label = 'CDR', color = 'green', marker = 'o')
plt.xlabel('Noise level')
plt.ylabel(r'$\sigma_{z}$')
plt.legend()
plt.show()

It is also important to explore convergence for the CDR and vnCDR methods, let have a look at how the accuracy of the solution changes with increasing training set size and number of non-Cliffords.

In [ ]:
M_2 = 10
M_3 = 20
print('Data regression underway')
vnCDR_mit_M_2, CDR_mit_M_2, noisy_M_2, _ = clifford_data_mitigation(circuit_new, scale_factors, M_2, N)
vnCDR_mit_M_3, CDR_mit_M_3, noisy_M_3, _ = clifford_data_mitigation(circuit_new, scale_factors, M_3, N)

Lets see how this looks on a graph:

In [ ]:
plt.figure(2)
plt.plot([M_2,M_3,M], [Abs_error(noisy_M_2, exact), Abs_error(noisy_M_3, exact), Abs_error(noisy, exact)],
         label = 'noisy', color = 'black')
plt.plot([M_2,M_3,M], [Abs_error(zne_lin, exact), Abs_error(zne_lin, exact), Abs_error(zne_lin, exact)],
         label = 'lin zne', color = 'black', ls='--')
plt.plot([M_2,M_3,M], [Abs_error(vnCDR_mit_M_2, exact), Abs_error(vnCDR_mit_M_3,exact), Abs_error(vnCDR_mit, exact)],
         label = 'vnCDR', color = 'red')
plt.plot([M_2,M_3,M], [Abs_error(CDR_mit_M_2, exact), Abs_error(CDR_mit_M_3,exact), Abs_error(CDR_mit, exact)],
         label = 'CDR', color = 'blue')
plt.legend()
plt.ylabel('Absolute error')
plt.xlabel('Training set size')
plt.tight_layout()
plt.show()

Convergence is observed with training set size. In general the CDR method converges more slowly. When converged both regression methods show an improvement over linear ZNE, but their performance is very similar. Due to the training set construction method, the convergence observed can be very quick or more slow, depending on how lucky one gets when completeing the Clifford projections. It can also be quite discontinous without doing any repeats, so the above graph may appear a little odd.

We should also see some convergence with the number of non-Cliffords, lets explore that:

In [ ]:
N_2 = 1
N_3 = 3
print('Data regression underway')
vnCDR_mit_N_2, CDR_mit_N_2, noisy_N_2, _ = clifford_data_mitigation(circuit_new, scale_factors,M, N_2)
vnCDR_mit_N_3, CDR_mit_N_3, noisy_N_3, _ = clifford_data_mitigation(circuit_new, scale_factors,M, N_3)

In [ ]:
plt.figure(3)
plt.plot([N_2,N_3,N], [Abs_error(noisy_N_2, exact), Abs_error(noisy_N_3, exact), Abs_error(noisy, exact)],
         label = 'noisy', color = 'black')
plt.plot([N_2,N_3,N], [Abs_error(zne_lin, exact), Abs_error(zne_lin, exact), Abs_error(zne_lin, exact)],
         label = 'lin zne', color = 'black', ls='--')
plt.plot([N_2,N_3,N], [Abs_error(vnCDR_mit_N_2, exact), Abs_error(vnCDR_mit_N_3,exact), Abs_error(vnCDR_mit, exact)],
         label = 'vnCDR', color = 'red')
plt.plot([N_2,N_3,N], [Abs_error(CDR_mit_N_2, exact), Abs_error(CDR_mit_N_3,exact), Abs_error(CDR_mit, exact)],
         label = 'CDR', color = 'blue')
plt.legend()
plt.ylabel('Absolute error')
plt.xlabel('Number non-Cliffords')
plt.tight_layout()
plt.show()

In the plots above convergence can be difficult to see clearly due to the random nature of the training set construction method. That makes for quite a lot of fluctuation in the quality of the mitgiation at various training set parameters. 